# 批标准化 BatchNormalization

batchNormalization 批标准化，与普通的数据标准化相似，是将分散的数据统一的一种做法，也是优化神经网络的一种方法，因为具有统一规格的数据能够让机器学习更容易学习到其中的规律


在神经网络中，数据分布会对训练产生影响


In [6]:
# 有时候激励函数对于变化范围较大的数据不敏感
import torch as t
x1 = t.tensor(5,dtype=t.float32)
x2 = t.tensor(40,dtype=t.float32)

print(t.tanh(x1),t.tanh(x2))

tensor(0.9999) tensor(1.)


可以看到，tanh(5)约等于tanh(40),这是很糟糕的，所以要在层与层之间把数据处理一下，让他们都在激活函数的敏感区间之内。

训练深层神经网络十分困难，特别是我们的目标是要在较短时间内使它收敛的时候，BN是一种有效的技术，可持续加速深层网络的收敛速度。

从形式上来说，BN根据以下表达式转换输入X

$$\mathrm{BN}(\mathbf{x}) = \boldsymbol{\gamma} \odot \frac{\mathbf{x} - \hat{\boldsymbol{\mu}}_\mathcal{B}}{\hat{\boldsymbol{\sigma}}_\mathcal{B}} + \boldsymbol{\beta}.$$

其中拉伸参数gamma和偏移参数beta他们的形状与输入X相同，但是是属于nn.parameters的，需要被学习

# 从头开始实现BN层

In [5]:
import torch as t
import torch.nn as nn
from torch import Tensor
def batch_norm(X:Tensor,gamma:Tensor,beta:Tensor,moving_mean:Tensor,moving_var:Tensor,eps:float,momentum:float)->tuple[Tensor,Tensor,Tensor]:
    # 通过is_grad_enabled 来判断当前模式是训练模式还是预测模式
    if not t.is_grad_enabled():
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat =(X-moving_mean)/t.sqrt(moving_var+eps)
    else:
        assert len(X.shape) in (2,4)
        
        # 如果使用全连接层，计算特征维上的均值和方差
        if len(X.shape)==2:
            mean = X.mean(dim=0)
            var = ((X-mean)**2).mean(dim = 0)
        # 如果使用卷积层，计算通道维上的均值和方差,即所有的某通道的数值和除以（行乘以列乘以batch数）
        else:
            mean = X.mean(dim=(0,2,3),keepdim=True)
            var = ((X-mean)**2).mean(dim = (0,2,3),keepdim=True)
        
        # 训练模式下，用当前的均值和方差做标准化
        X_hat =(X-mean)/t.sqrt(var+eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum*moving_mean +(1-momentum)*mean
        moving_var = momentum*moving_var +(1-momentum)*var

    # 缩放和移位
    Y=gamma*X_hat+beta
    return Y,moving_mean.data,moving_var.data
    

# 自定义BN模块

In [15]:
class BatchNorm(nn.Module):
    def __init__(self,num_features:int,num_dims:int)->None:
        super().__init__()

        if num_dims==2:
            shape =(1,num_features)
        else:
            shape = (1,num_features,1,1)
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成1和0
        self.gamma = nn.Parameter(t.ones(shape))
        self.beta =nn.Parameter(t.zeros(shape))
        # 非模型参数的变量初始化为0和1
        self.moving_mean = t.zeros(shape)
        self.moving_var = t.ones(shape )

    def forward(self,X:Tensor)->Tensor:
        if self.moving_mean.device!=X.device:
            self.moving_mean=self.moving_mean.to(X.device)
            self.moving_var=self.moving_var.to(X.device)
        #保存更新之后的moving_mean和moving_var
        Y,self.moving_mean,self.moving_var=batch_norm(X,self.gamma,self.beta,self.moving_mean,self.moving_var,eps=1e-5,momentum=0.9)
        return Y

In [25]:
a= t.tensor([[1.,2.,3.]])
label = t.tensor([0])
loss_func =t.nn.CrossEntropyLoss()
loss_crossepy=loss_func(a,label)

In [26]:
loss_crossepy

tensor(2.4076)

In [24]:
import torch.nn.functional as F
#########################
a_softmax = F.softmax(a,1)
log_a = a_softmax.log()
loss_nll = F.nll_loss(log_a,label)
#########################
loss_nll

tensor(2.4076)

In [5]:
import torch as t
import torch.nn.functional as F
a =t.tensor([[0.,1.,1.]])
label=t.tensor([2])
a_softmax = F.softmax(a,1)
a_softmax


tensor([[0.1554, 0.4223, 0.4223]])